# Imports

In [ ]:
# pdf scanner
import PyPDF2
from pypdf import PdfReader
import pdfminer
# arrays
import pandas as pd
import numpy as np
# os
import os
# NLP
    # regex
import re
    # unicode
import unidecode

## ✅ Pulling Images (only required once)

In [ ]:
# reader object
# reader = PdfReader('data.pdf')

In [ ]:
# number of pages within data pdf
# num_pages = len(reader.pages)

In [ ]:
# images pulled from pdf
# reader = PdfReader("data.pdf")
# for page in reader.pages:
#     for image in page.images:
#         with open(image.name, "wb") as fp:
#             fp.write(image.data)

# Retrieving data from all pdfs

## Not using isolated pdf version of retrieval

### Proceeding with compiled PDF, code remains for documentation purposes

In [ ]:
# using pdfs seperated for \n inside the text

# importing required modules
# bodies = []
# names = ['dali','warhol','banksy','marcchagall','picasso','rembrandt','kaws','leoneardtsuguharufoujita','yayoi-kusama','zhangdaqian']
# # creating a pdf file object
# for name in names:
#     open_name = './' + name + '.pdf'
#     pdfFileObj = open(open_name, 'rb')
#     # creating a pdf reader object
#     pdfReader = PyPDF2.PdfReader(pdfFileObj)
#     # printing number of pages in pdf file
#     num_pages = len(pdfReader.pages)
#     print(f"Number of pages: {num_pages}")
#     # loop through all pages and extract text
#     for page_num in range(num_pages):
#         pageObj = pdfReader.pages[page_num]
#         text = pageObj.extract_text()
#         bodies.append(text)
#         print(f"Page {page_num+1}:\n{name}")
#         # validate the process body of text is correct format
#         if type(text) is str:
#             print('Successfully pulled text from page.\n')
#         else:
#             print('Failed to pull text.')
#     # closing the pdf file object
#     pdfFileObj.close()
#     print(f'+++++++++++++++++++++\nFinished text from {name} pdf\n+++++++++++++++++++++\n')


In [ ]:
# using as a whole pdf because it removes the line breaks \n

# importing required modules
bodies = []

# creating a pdf file object
pdfFileObj = open('./data.pdf', 'rb')

# creating a pdf reader object
pdfReader = PyPDF2.PdfReader(pdfFileObj)

# printing number of pages in pdf file
num_pages = len(pdfReader.pages)
print(f"Number of pages: {num_pages}")

    # loop through all pages and extract text
for page_num in range(num_pages):
    pageObj = pdfReader.pages[page_num]
    text = pageObj.extract_text()
    bodies.append(text)
    print(f"Page {page_num+1}:\ndata.pdf")
    # validate the process body of text is correct format
    if type(text) is str:
        print('Successfully pulled text from page.\n')
    else:
        print('Failed to pull text.')
# closing the pdf file object
pdfFileObj.close()


In [ ]:
# function for isolating and pulling jp & ch characters
def remove_ch_jp_characters(text):
  """
  Removes all chinese characters from a string
  """
  pattern = re.compile(r'[\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff\uff66-\uff9f]')
  return pattern.sub('', text)

In [ ]:
# decoding letters with accents into regular letters & removing character symbols
body_clean = []
for body in bodies:
    body = unidecode.unidecode(body)
    # removing the title of pdf from the body of text
    body = body.replace('Price Database31 October 2023','')
    body_clean.append(body)

In [ ]:
# separating each entry by artist name
names = ['Zhang Daqian', 'Andy Warhol', 'Banksy', 'Salvador Dali','Marc Chagell','Pablo Picasso',
         'Rembrandt van Rijn','KAWS','Leonard Tsuguharu Foujita','Yayoi Kusama']
# container for all entries
new_body = []
# container for each artist that has multiple names in the entry
banksy_clean = []
marc_clean = []
pablo_clean = []
rembrandt_clean = []
dali_clean = []
for body in body_clean:
    for name in names:

        # isolating the entries that contain multiple counts of name in entry
            # the text in the if statement is the combined text where the name is repeated
            # these varie between title of piece or within the lot description

        # Banksy / KAWS
        if name == 'Banksy' or 'KAWS':
            if 'Only Banksy' in body:
                banksy_clean.append(body)
            elif 'Banksy and 21st Century Editions' in body:
                banksy_clean.append(body)

        # Marc Chagell
        if name == 'Marc Chagell':
            if 'Marc Chagall, Colour of Life' in body:
                marc_clean.append(body)
        # Dali
        if name == 'Salvador Dali':
            if 'After Salvador Dali' in body:
                dali_clean.append(body)
            elif 'SALVADOR DALI\' "Genesis"' in body:
                dali_clean.append(body)
            elif 'Currier & Ives as Interpreted by Salvador Dali' in body:
                dali_clean.append(body)
        # Picasso
        if name == 'Pablo Picasso':
            if 'After Pablo Picasso' in body:
                pablo_clean.append(body)
        # Rembrandt
        if name == 'Rembrandt van Rijn':
            if 'School of Rembrandt van Rijn' in body:
                rembrandt_clean.append(body)
            elif 'Circle of Rembrandt van Rijn' in body:
                rembrandt_clean.append(body)
            elif 'Studio of Rembrandt van Rijn' in body:
                rembrandt_clean.append(body)
            elif 'Follower of Rembrandt van Rijn' in body:
                rembrandt_clean.append(body)

        # All other values that do not have repeated names
        else:
            entries = body.split(f"{name}")[1:]
        for entry in entries:
            entry = entry.strip()
            # adding an additional space back in between the name and piece_name
            a = ''.join([name+' ',entry])
            a = a.replace('\x00','-')
            new_body.append(a)
            print(a)
            print("="*30)


In [ ]:
# Removing entries from new_body that do not have complete data
new_body = [body for body in new_body if len(body) >= 130]

## Isolated Lists of Entries that have the name of artist within the data.

### Banksy / KAWS

In [ ]:
# removing duplicates
banksy_clean = list(set(banksy_clean))

In [ ]:
banksy_ = []
for i in banksy_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # adding space after artist name for ease of regex
        entry = entry.replace('Banksy','Banksy ')
        entry = entry.replace('KAWS','KAWS ')
        # removing double spaces
        entry = entry.replace('  ', ' ')
        banksy_.append(entry)

# cleaned banksy entries with multiple appearances of names
banksy_

### Rembrandt

In [ ]:
# removing duplicates
rembrandt_clean = list(set(rembrandt_clean))

In [ ]:
rembrandt_ = []
for i in rembrandt_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # adding space after artist name for ease of regex
        entry = entry.replace('Rembrandt van Rijn','Rembrandt van Rijn ')
        # removing double spaces
        entry = entry.replace('  ', ' ')
        rembrandt_.append(entry)

# cleaned rembrandt entries with multiple appearances of names
rembrandt_

### Marc Chagall

In [ ]:
marc_ = []
for i in marc_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # adding space after artist name for ease of regex
        entry = entry.replace('Marc Chagall','Marc Chagall ')
        # removing double spaces
        entry = entry.replace('  ', ' ')
        marc_.append(entry)

# cleaned marc chagall entries with multiple appearances of names
marc_

### Pablo Picasso

In [ ]:
pablo_ = []
for i in pablo_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # adding space after artist name for ease of regex
        entry = entry.replace('Pablo Picasso','Pablo Picasso ')
        # removing double spaces
        entry = entry.replace('  ', ' ')
        pablo_.append(entry)

# cleaned pablo picasso entries with multiple appearances of names
pablo_

### Salvador Dali

In [ ]:
dali_clean

In [ ]:
dali_ = []
for i in dali_clean:
    # splitting by new line
    entries = i.split('\n')
    for entry in entries:
        # adding space after artist name for ease of regex
        entry = entry.replace('Dali','Dali ')
        # removing double spaces
        entry = entry.replace('  ', ' ')
        dali_.append(entry)

# cleaned dali entries with multiple appearances of names
dali_ = dali_[1:]
dali_

In [ ]:
# combing all cleaned entries
add_ins = dali_ + rembrandt_ + marc_ + pablo_ + banksy_
len(add_ins)

## Connecting multiple name counts with the rest of values that did not need extra cleaning

In [ ]:
print(len(new_body))
new_body.extend(add_ins)
print(len(new_body))

In [ ]:
# reaffirming all cleaned entries are removed 
new_body = [body for body in new_body if len(body) >= 130]

## Isolating values by regex needs

In [ ]:
# pre regex
# 1. if name count < 1, drop value (DONE)

# regex
    # 2. Make depth optional
    # 3. make foreign currency (estimate & hammer price) optional
    # 4.

# vars
# new_body = collective of all values
# has_fc
    # fc_has_depth
    # fc_no_depth

In [ ]:
# seperating by IF has foreign currency
foreign_currency = ['HKD','EUR','NZD','AUD','GBP','KRW','CNY','JPY','CHF','SGD','CAD','TWD','INR','THB','RUB','SEK','DKK','NOK',
                    'PLN','ZAR','MXN','BRL','TRY','AED','SAR','QAR','KWD','OMR','BHD','JOD','ILS','EGP','IDR','MYR','PHP','CZK',
                    'HUF','CLP','ARS','COP','PEN']
# seperating by IF has depth
# Depth as a keyword

In [ ]:
has_foreign_currency = [body for body in new_body if any(currency in body for currency in foreign_currency)]
len(has_foreign_currency)

## Manual fixes for names that were not separated for one reason or another

In [ ]:
has_foreign_currency

In [ ]:
for i, body in enumerate(has_foreign_currency):
    if '\nSalvador Dali' in body:
        print(f'Place in List: {i}')
        body = body.replace('\nSalvador Dali','Salvador Dali')

In [ ]:
has_foreign_currency

In [ ]:
has_depth = [body for body in new_body if 'Depth' in body]

In [ ]:
len(has_depth)

In [ ]:
depth_has_fc = [body for body in has_depth if any(fc in body for fc in foreign_currency)]
depth_no_fc = [body for body in has_depth if not any(fc in body for fc in foreign_currency)]

In [ ]:
print(len(depth_has_fc), len(depth_no_fc))